In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

### 크롤링에 앞서 필요한 것들

In [5]:
#headers에 user agent 정보 넣기
headers = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"}
url = 'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&page='

In [6]:
#request.get()으로 연결
r =requests.get(url, headers=headers)
#status_code 응답확인하기 (200이면됨)
r.status_code

200

In [8]:
#beautifulsoup로 접속하기
soup=BeautifulSoup(r.text, 'html.parser') #text 또는 content

#### 크롤링을 하기에 앞서 우리가 필요한 데이터들이 어디에 담겨있었는지 확인
#### 해당 사이트의 경우엔 odd와 even안에 있었음

In [11]:
#선수 정보가 담긴 태그나 클래스 찾기
player_info = soup.find_all('tr', class_=['even','odd'])
#그리고 td안에 모든 정보가 차례로 있었음 나이, 국적, 포지션 등등

In [12]:
#첫번째 요소 파악하기
player_info[0]

<tr class="odd">
<td class="zentriert">1</td><td class=""><table class="inline-table"><tr><td rowspan="2"><a href="#"><img alt="Kylian Mbappé" class="bilderrahmen-fixed" src="https://img.a.transfermarkt.technology/portrait/small/342229-1632686046.jpg?lm=1" title="Kylian Mbappé"/></a></td><td class="hauptlink"><a href="/kylian-mbappe/profil/spieler/342229" title="Kylian Mbappé">Kylian Mbappé</a></td></tr><tr><td>Centre-Forward</td></tr></table></td><td class="zentriert">24</td><td class="zentriert"><img alt="France" class="flaggenrahmen" src="https://tmssl.akamaized.net/images/flagge/verysmall/50.png?lm=1520611569" title="France"/></td><td class="zentriert"><a href="/fc-paris-saint-germain/startseite/verein/583" title="Paris Saint-Germain"><img alt="Paris Saint-Germain" class="" src="https://tmssl.akamaized.net/images/wappen/verysmall/583.png?lm=1522312728" title="Paris Saint-Germain"/></a></td><td class="rechts hauptlink"><a href="/kylian-mbappe/marktwertverlauf/spieler/342229">€180.00

In [13]:
#전체 개수 파악하기
len(player_info)

25

### 크롤링 설계
#### 1단계: tr태그의 클래스가 odd와 even인 것을 find_all을 이용해 모두 찾음
#### 2단계: 필요한 정보를 담을 리스트를 생성
#### 3단계: 반복문을 이용해 td인 것만 모두 찾음
#### 4단계: append를 이용해 리스트에 요소들을 추가 (ex; number.append(player[0].text))
#### 5단계: 페이지의 규칙을 발견해 반복문을 이용해 여러 페이지를 크롤링할 수 있도록 구현
#### 6단계: 크롤링한 텍스트들을 데이터프레임에 저장

## 여러 페이지를 크롤링하는 방법

In [4]:
#headers에 user agent 정보 넣기
headers = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"}

#정보를 담을 리스트 생성
number=[]
name=[]
position=[]
age=[]
nation=[]
team=[]
value=[]

#url 주소 넣기 (원하는 페이지 수만큼 넣기)
for i in range(1, 4):
    url = f"https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&page={i}"

    #request.get()으로 연결
    r =requests.get(url, headers=headers)
    #status_code 응답확인하기 (200이면됨)
    r.status_code

    #beautifulsoup로 접속하기
    soup=BeautifulSoup(r.text, 'html.parser') #text 또는 content

    #선수 정보가 담긴 태그나 클래스 찾기
    player_info = soup.find_all('tr', class_=['even','odd'])
    
    # player_info에서 'td'태그만 모두 찾기
    for info in player_info:
        player = info.find_all("td")

        number.append(player[0].text)
        name.append(player[3].text)
        position.append(player[4].text)
        age.append(player[5].text)
        nation.append(player[6].img['alt'])
        team.append(player[7].img['alt'])
        value.append(player[8].text.strip())
        
    time.sleep(1)
    df = pd.DataFrame(
        {"number":number,
         "name":name,
         "position":position,
         "age":age,
         "nation":nation,
         "team":team,
         "value":value}
    )
df
    

,number,name,position,age,nation,team,value
0,1,Kylian Mbappé,Centre-Forward,23,France,Paris Saint-Germain,€160.00m
1,2,Erling Haaland,Centre-Forward,22,Norway,Manchester City,€150.00m
2,3,Vinicius Junior,Left Winger,22,Brazil,Real Madrid,€100.00m
3,4,Phil Foden,Central Midfield,22,England,Manchester City,€90.00m
4,5,Mohamed Salah,Right Winger,30,Egypt,Liverpool FC,€90.00m
...,...,...,...,...,...,...,...
70,71,Youri Tielemans,Central Midfield,25,Belgium,Leicester City,€55.00m
71,72,Virgil van Dijk,Centre-Back,31,Netherlands,Liverpool FC,€55.00m
72,73,David Alaba,Centre-Back,30,Austria,Real Madrid,€55.00m
73,74,Ronald Araújo,Centre-Back,23,Uruguay,FC Barcelona,€50.00m


In [5]:
player_info = soup.find_all('tr', class_=['even','odd'])
player_info

[<tr class="odd">
 <td class="zentriert">51</td><td class=""><table class="inline-table"><tr><td rowspan="2"><a href="#"><img alt="Ansu Fati" class="bilderrahmen-fixed" src="https://img.a.transfermarkt.technology/portrait/small/466810-1560761660.jpg?lm=1" title="Ansu Fati"/></a></td><td class="hauptlink"><a href="/ansu-fati/profil/spieler/466810" title="Ansu Fati">Ansu Fati</a></td></tr><tr><td>Left Winger</td></tr></table></td><td class="zentriert">19</td><td class="zentriert"><img alt="Spain" class="flaggenrahmen" src="https://tmssl.akamaized.net/images/flagge/verysmall/157.png?lm=1520611569" title="Spain"/><br/><img alt="Guinea-Bissau" class="flaggenrahmen" src="https://tmssl.akamaized.net/images/flagge/verysmall/60.png?lm=1520611569" title="Guinea-Bissau"/></td><td class="zentriert"><a href="/fc-barcelona/startseite/verein/131" title="FC Barcelona"><img alt="FC Barcelona" class="" src="https://tmssl.akamaized.net/images/wappen/verysmall/131.png?lm=1406739548" title="FC Barcelona"/>

## 빈 리스트에 각 정보를 함수를 이용해 담기

In [ ]:
#정보 안에서 td에 해당하는 애들의 정보를 가져오고 그것들의 n번째에 어떤 정보들이 담겨있는지 파악 후, 필요한 정보들만 가져옴 
for info in player_info:
    player = info.find_all('td')
    print(player[0])

In [ ]:
for info in player_info:
    player = info.find_all('td')
    number.append(player[0].text)
    name.append(player[3].text)
    position.append(player[4].text)
    age.append(player[5].text)
    nation.append(player[6].img['alt'])   #image 태그에 alt 속성에 들어있는 데이터면 얘처럼 추가
    team.append(player[7].img['alt'])
    value.append(player[8].text.strip()) #앞뒤에 공백이 있으면 strip()이용해서 제거

In [ ]:
#데이터프레임에 저장하기 
df = pd.DataFrame(
        {"number":number,
         "name":name,
         "position":position,
         "age":age,
         "nation":nation,
         "team":team,
         "value":value}
    )
df

In [3]:
df.to_csv('transfermarkt50.csv', index=False)

In [7]:
transfer = pd.read_csv("C:/Users/user/Desktop/transfermarkt50.csv")
transfer

,number,name,position,age,nation,team,value
0,1,Kylian Mbappé,Centre-Forward,23,France,Paris Saint-Germain,€160.00m
1,2,Erling Haaland,Centre-Forward,22,Norway,Manchester City,€150.00m
2,3,Vinicius Junior,Left Winger,22,Brazil,Real Madrid,€100.00m
3,4,Phil Foden,Central Midfield,22,England,Manchester City,€90.00m
4,5,Mohamed Salah,Right Winger,30,Egypt,Liverpool FC,€90.00m
...,...,...,...,...,...,...,...
95,96,Jonathan David,Centre-Forward,22,Canada,LOSC Lille,€45.00m
96,97,Raphinha,Right Winger,25,Brazil,FC Barcelona,€45.00m
97,98,Ferran Torres,Left Winger,22,Spain,FC Barcelona,€45.00m
98,99,Martin Ødegaard,Attacking Midfield,23,Norway,Arsenal FC,€45.00m
